In [33]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [34]:
data = pd.read_csv("../Code/Dataset/mpi_roof.csv", encoding='latin-1')
data['Date Time'] = pd.to_datetime(data['Date Time'], format='%d.%m.%Y %H:%M:%S')
data = data.set_index("Date Time")

In [35]:
is_nan = data.isnull().values.any()
if (is_nan):
    print("There exist null values")
else:
    print("No null values")

No null values


In [36]:
#Check if exist a case when having no record of rain (0mm) but the raining(s)
checker =  not data.loc[(data['rain (mm)'] == 0) & (data['raining (s)'] > 0), ['rain (mm)', 'raining (s)']].empty
if (checker):
    print("There exist a case when having no record of rain (0mm) but the raining(s)")
else:
    print("There is no case when having no record of rain (0mm) but the raining(s)")


There exist a case when having no record of rain (0mm) but the raining(s)


In [37]:
#Check if exist a case when having no record of raining (s) but rain
checker_2 = not data.loc[(data['rain (mm)'] > 0) & (data['raining (s)'] == 0), ['rain (mm)', 'raining (s)']].empty
if (checker_2):
    print("There exist a case")
else:
    print("There is no case")

There exist a case


In [38]:
data['Is Rain'] = np.where((data['rain (mm)'] > 0) & (data['raining (s)'] > 0), 1, 0)
data.head()


,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),...,max. wv (m/s),wd (deg),rain (mm),raining (s),SWDR (W/m²),PAR (µmol/m²/s),max. PAR (µmol/m²/s),Tlog (degC),CO2 (ppm),Is Rain
Date Time,,,,,,,,,,,,,,,,,,,,,
2024-01-01 00:10:00,979.65,7.63,282.43,2.83,71.53,10.47,7.49,2.98,4.77,7.64,...,5.04,209.0,0.0,0.0,0.0,0.0,0.0,15.38,428.8,0
2024-01-01 00:20:00,979.56,7.63,282.45,2.76,71.18,10.47,7.45,3.02,4.75,7.61,...,4.50,204.5,0.0,0.0,0.0,0.0,0.0,15.42,428.1,0
2024-01-01 00:30:00,979.51,7.56,282.37,2.66,71.00,10.42,7.40,3.02,4.71,7.55,...,6.96,208.1,0.0,0.0,0.0,0.0,0.0,15.47,428.3,0
2024-01-01 00:40:00,979.43,7.51,282.33,2.59,70.91,10.38,7.36,3.02,4.69,7.52,...,5.13,201.8,0.0,0.0,0.0,0.0,0.0,15.52,427.8,0
2024-01-01 00:50:00,979.35,7.42,282.25,2.51,70.93,10.32,7.32,3.00,4.66,7.47,...,4.21,198.6,0.0,0.0,0.0,0.0,0.0,15.53,428.0,0


In [39]:
#Resample data
def round_mean(value):
    if 0 < value < 1:
        return 1
    return value

resample_data = data.resample('6h').mean()
resample_data['Is Rain'] = resample_data['Is Rain'].apply(round_mean)


In [40]:
#Preprocessing data
scaler = StandardScaler()
new_data = resample_data.drop(columns=['Is Rain'])
scaled_data = scaler.fit_transform(new_data)
scaled_data = pd.DataFrame(scaled_data, columns=new_data.columns, index=new_data.index)
scaled_data['Is Rain'] = resample_data['Is Rain']
scaled_data.head()

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),...,max. wv (m/s),wd (deg),rain (mm),raining (s),SWDR (W/m²),PAR (µmol/m²/s),max. PAR (µmol/m²/s),Tlog (degC),CO2 (ppm),Is Rain
Date Time,,,,,,,,,,,,,,,,,,,,,
2024-01-01 00:00:00,-1.038910,-0.607535,-0.495156,-0.753116,-0.172449,-0.683671,-0.816243,-0.354583,-0.797870,-0.798365,...,1.667840,0.422569,-0.245925,-0.363071,-0.726738,-0.728010,-0.722254,-0.796365,-0.511887,0.0
2024-01-01 06:00:00,-0.977220,-0.653649,-0.546302,-0.738430,-0.025738,-0.713803,-0.806097,-0.410650,-0.788777,-0.788717,...,2.106938,0.467198,-0.245925,-0.363071,-0.434004,-0.448788,-0.438287,-0.855740,-0.505469,0.0
2024-01-01 12:00:00,-0.864557,-0.569601,-0.474746,-0.667503,-0.089449,-0.657749,-0.752443,-0.369990,-0.736995,-0.737199,...,2.126210,0.472289,-0.245925,-0.213243,-0.343666,-0.345776,-0.339271,-0.715571,-0.566171,0.0
2024-01-01 18:00:00,-0.635749,-0.875138,-0.796193,-0.725994,0.610058,-0.850614,-0.795767,-0.632994,-0.783864,-0.784152,...,0.457293,0.534779,-0.107249,0.297534,-0.726738,-0.728010,-0.722254,-0.930570,-0.437895,1.0
2024-01-02 00:00:00,-0.864705,-0.928105,-0.826135,-0.643314,0.972406,-0.882210,-0.734726,-0.735488,-0.719308,-0.719306,...,-0.208915,0.074179,-0.107249,1.073915,-0.726738,-0.728010,-0.722254,-0.905752,-0.195250,1.0
